In [16]:
!pip install shap lime torch transformers sentencepiece numpy


In [25]:
from sentencepiece import SentencePieceProcessor

# Load SentencePiece tokenizer
spm = SentencePieceProcessor(model_file="C:/Users/hp/AmharicSPM.model")


In [26]:
import torch

# Define a function to tokenize text inputs using the SentencePiece tokenizer
def tokenize_input(texts, max_length=128):
    # Encode each text using the SentencePiece tokenizer
    inputs = [spm.encode(text, out_type=int) for text in texts]
    
    # Pad sequences to a fixed length
    padded_inputs = torch.tensor([i + [0] * (max_length - len(i)) if len(i) < max_length else i[:max_length] for i in inputs])
    
    return padded_inputs


In [ ]:
import shap

# Define a wrapper function for the tokenizer to make it compatible with SHAP
def amharic_tokenizer(texts):
    # Tokenize the texts
    return tokenize_input(texts)

# Initialize the SHAP explainer with the model and tokenizer
explainer = shap.Explainer(peft_model, amharic_tokenizer)

# Select a sample of texts to explain
sample_texts = test_texts

# Compute SHAP values for the sample texts
shap_values = explainer(sample_texts)

# Visualize the SHAP values
shap.summary_plot(shap_values)


In [ ]:
from lime.lime_text import LimeTextExplainer

# Initialize LIME Text Explainer
explainer = LimeTextExplainer(class_names=["Negative", "Positive"])

# Define a function to predict class probabilities
def predict_proba(texts):
    tokenized_inputs = tokenize_input(texts)
    with torch.no_grad():
        outputs = peft_model(tokenized_inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
    return probs.cpu().numpy()

# Select a text to explain
text_to_explain = "እትምስክርነው ነበር፤ ነገር ግን አላመለከተም።"

# Explain the prediction using LIME
exp = explainer.explain_instance(text_to_explain, predict_proba, num_features=10)

# Show the explanation
exp.show_in_notebook()
